# Web Scraping LinkedIn Job Postings (Authenticated LinkedIn Account)

## File Imports and Set-Up

In [1]:
pip install install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
import time
import pandas as pd
import os

#using regular expressions to find the first number
import re

from dotenv import load_dotenv

#loading env variables
load_dotenv()

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

url1 = 'https://www.linkedin.com/jobs/search/?currentJobId=3705320512&f_E=2&f_WT=2&keywords=flutter&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true'

driver = webdriver.Chrome()
driver.get(url1)
driver.implicitly_wait(10)

## Logging in to LinkedIn Using a Dummy Account

In [3]:
signin_button = driver.find_element(By.CLASS_NAME, 'nav__button-secondary')
signin_button.click()

username = driver.find_element(By.ID, 'username')
username.send_keys(os.environ.get('EMAIL'))

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ.get('PASSWORD'))

login_button = driver.find_element(By.CLASS_NAME, 'btn__primary--large')
login_button.click()

## Counting the Number of Jobs

In [4]:
y = driver.find_elements(By.CLASS_NAME, 'jobs-search-results-list__subtitle')[0].text

try:
    if y.endswith('+') and "," in y:
        cleaned_y = y[:-1].replace(",", "")

    else:
        cleaned_y = y
        
    n = pd.to_numeric(cleaned_y)
    
except:
    parts = y.split()

    n = int(parts[0])

n

34

## Collecting the Links for Each Job Posting

In [5]:
# Get all links for these offers
links = []

# Navigate 13 pages
print('Links are being collected now.')
try:
    for page in range(2, 16):
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                href = a.get_attribute('href')
                if href and href.startswith("https://www.linkedin.com/jobs/view") and href not in links:
                    links.append(href)

            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)

        print(f'Collecting the links on page: {page - 1}')
        # go to next page:
        next_page_button = driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']")
        driver.execute_script("arguments[0].click();", next_page_button)
        time.sleep(3)
except Exception as e:
    print(e)

print(f'Found {len(links)} links for job offers')

Links are being collected now.
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@aria-label='Page 3']"}
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000107897e08 chromedriver + 5025288
1   chromedriver                        0x000000010788ec23 chromedriver + 4987939
2   chromedriver                        0x0000000107430e67 chromedriver + 409191
3   chromedriver                        0x00000001074801b9 chromedriver + 733625
4   chromedriver                        0x0000000107480371 chromedriver + 734065
5   chromedriver                        0x00000001074c6194 chromedriver + 1020308
6   chromedriver                        0x00000001074a750d chromedriver + 894221
7   chromedriver                        0x00000001074c3571 chromedriver + 1009009
8   c

## Scraping Proper

In [6]:
# Create empty lists to store information
job_titles = []
company_names = []
company_location = []
post_dates = []
applicants = []
skills_list = []
job_desc = []

# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i, link in enumerate(links):
    try:
        driver.get(link)
        time.sleep(2)
        
        # Click See More Button
        see_more_button = driver.find_element(By.XPATH, f"//button[@aria-label='Click to see more description']")
        driver.execute_script("arguments[0].click();", see_more_button)
        time.sleep(2)
    except Exception as e:
        print(e)
    
    # Find the general information of the job offers
    contents = driver.find_elements(By.CLASS_NAME, 'p5')
    for content in contents:
        try:
            title = content.find_element(By.CLASS_NAME, 'job-details-jobs-unified-top-card__job-title').text
            job_titles.append(title)

            details = content.find_element(By.CLASS_NAME, 'job-details-jobs-unified-top-card__primary-description').text.split('·')
            print(details)   
            
            company_names.append(details[0])
            
            # Extract company_location and post_dates directly
            company_location_and_post_date = details[1].strip()
            company_location.append(company_location_and_post_date)
            
            # Split company_location_and_post_date into location and post_dates
            split_details = re.split(r'(\d+)', company_location_and_post_date)
            split_details = [part.strip() for part in split_details if part.strip()]
            
            print("Split details: ", split_details)
            
            # Separate lists for the extracted items
            if len(split_details) == 3:
                company_location.append(split_details[0])
                
                combining_post_dates = split_details[1] + split_details[2]
                
                #post_dates.append(split_details[1])
                post_dates.append(combining_post_dates)
            else:
                # Handle the case where there's no post date
                company_location.append(split_details[0])
                post_dates.append("N/A")
            
            applicants.append(details[2])
            
            #Extracting skills for the job
            skills_details = content.find_element(By.CLASS_NAME, 'job-details-jobs-unified-top-card__job-insight-text-button').text.split(',')
            #skills = skills_details.pop()
            print("Skills: ", skills_details)
            
            #Using list comprehension to remove Skills: from the list
            cleaned_skills_list = [skill.replace('Skills: ', '') for skill in skills_details[:-1]]
            skills_list.append(cleaned_skills_list)
            
            
            print(company_names)
            print(company_location)
            print(post_dates)
            print(applicants)
            print(skills_list)
            
            print(f'Scraping the Job Offer {i + 1} DONE.')
        
        except Exception as e:
            print(e)
            
        time.sleep(2)
        
    
    # Scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME, 'jobs-description__content')
    for description in job_description:
        try:
            job_text = description.find_element(By.CLASS_NAME, "jobs-box__html-content").text
            job_desc.append(job_text)
            print(f'Scraping the Job Description {i + 1}')
        
        except Exception as e:
            print(e)
        
        time.sleep(2)

# Creating the dataframe
df = pd.DataFrame(list(zip(job_titles, company_names, company_location, post_dates, applicants, skills_list, job_desc)), columns=['Job Title', 'Company Name', 'Company Location', 'Post Dates', 'Number of Applicants', 'Skills List', 'Job Description'])

Visiting the links and collecting information just started.
['Sense Software Solutions ', ' Philippines Reposted  2 weeks ago  ', ' 88 applicants']
Split details:  ['Philippines Reposted', '2', 'weeks ago']
Skills:  ['Skills: Qlik Sense', ' Dashboards', ' +8 more']
['Sense Software Solutions ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted']
['2weeks ago']
[' 88 applicants']
[['Qlik Sense', ' Dashboards']]
Scraping the Job Offer 1 DONE.
Scraping the Job Description 1
['Career Shepherd ', ' Manila, National Capital Region, Philippines  8 minutes ago  ', ' 1 applicant']
Split details:  ['Manila, National Capital Region, Philippines', '8', 'minutes ago']
Skills:  ['Skills: Cascading Style Sheets (CSS)', ' JavaScript', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Manila, National Capital Region, Philippines']
['2weeks ago', '8minutes a

Scraping the Job Description 7
['Sourcepass ', ' Metro Manila  2 months ago  ', ' 125 applicants']
Split details:  ['Metro Manila', '2', 'months ago']
Skills:  ['Skills: Computer Science', ' Android Studio', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Manila, National Capital Region, Philippines', 'Philippines  23 hours ago', 'Philippines', 'Quezon City, National Capital Region, Philippines  1 week ago', 'Quezon City, National Capital Region, Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago', 'Manila, National Capital Region, Philippines', 'Metro Manila  1 month ago', 'Metro Manila', 'Metro Manila  2 months ago', 'Metro Manila']
['2weeks ago', '8minutes ago', '23hours ago', '1

Scraping the Job Description 12
['3Cloud ', ' Philippines Reposted  1 week ago  ', ' 237 applicants']
Split details:  ['Philippines Reposted', '1', 'week ago']
Skills:  ['Skills: Site Reliability Engineering', ' Bash', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', 'Vita Green ', 'VIPRE Security Group ', '3Cloud ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Manila, National Capital Region, Philippines', 'Philippines  23 hours ago', 'Philippines', 'Quezon City, National Capital Region, Philippines  1 week ago', 'Quezon City, National Capital Region, Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago', 'Manila, National Capital Region, Philippines', 'Metro Manila  1 month ago', 'Me

Message: no such element: Unable to locate element: {"method":"css selector","selector":".job-details-jobs-unified-top-card__job-title"}
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000107897e08 chromedriver + 5025288
1   chromedriver                        0x000000010788ec23 chromedriver + 4987939
2   chromedriver                        0x0000000107430e67 chromedriver + 409191
3   chromedriver                        0x00000001074801b9 chromedriver + 733625
4   chromedriver                        0x0000000107480371 chromedriver + 734065
5   chromedriver                        0x0000000107473b66 chromedriver + 682854
6   chromedriver                        0x00000001074a750d chromedriver + 894221
7   chromedriver                        0x0000000107473a28 chromedriver + 682536
8   chromedriver

Scraping the Job Description 18
['3Cloud ', ' Philippines Reposted  1 week ago  ', ' 468 applicants']
Split details:  ['Philippines Reposted', '1', 'week ago']
Skills:  ['Skills: Quality Assurance', ' Test Cases', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', 'Vita Green ', 'VIPRE Security Group ', '3Cloud ', 'OffSec ', 'Asurion ', 'WebFX ', '3Cloud ', 'Security Bank Corporation ', '3Cloud ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Manila, National Capital Region, Philippines', 'Philippines  23 hours ago', 'Philippines', 'Quezon City, National Capital Region, Philippines  1 week ago', 'Quezon City, National Capital Region, Philippines', 'Philippines  2 months ago', 'Philippines', 'Manila, National Capital Region, Philippines  1 month ago', 

...

Scraping the Job Description 32
['WebFX ', ' Philippines Reposted  3 hours ago  ', ' 1,314 applicants']
Split details:  ['Philippines Reposted', '3', 'hours ago']
Skills:  ['Skills: Takes Direction Well', ' Internet Access', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', 'Vita Green ', 'VIPRE Security Group ', '3Cloud ', 'OffSec ', 'Asurion ', 'WebFX ', '3Cloud ', 'Security Bank Corporation ', '3Cloud ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Michael Page ', 'Vita Green ', 'Quantrics Enterprises Inc. ', 'Inspectorio ', 'Nextiva ', 'RealPage, Inc. ', 'Netrix Global ', 'OffSec ', 'GoTeam ', 'WebFX ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Manila, National Capital Region, Philip

Scraping the Job Description 34
['WebFX ', ' Philippines Reposted  2 weeks ago  ', ' 120 applicants']
Split details:  ['Philippines Reposted', '2', 'weeks ago']
Skills:  ['Skills: Web Maintenance', ' Takes Direction Well', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', 'Vita Green ', 'VIPRE Security Group ', '3Cloud ', 'OffSec ', 'Asurion ', 'WebFX ', '3Cloud ', 'Security Bank Corporation ', '3Cloud ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Michael Page ', 'Vita Green ', 'Quantrics Enterprises Inc. ', 'Inspectorio ', 'Nextiva ', 'RealPage, Inc. ', 'Netrix Global ', 'OffSec ', 'GoTeam ', 'WebFX ', '3Cloud ', 'WebFX ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Manila, National Cap

Scraping the Job Description 35
['MultiplyMii ', ' Philippines  1 week ago  ', ' 49 applicants']
Split details:  ['Philippines', '1', 'week ago']
Skills:  ['Skills: NetSuite', ' Data Import/export', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', 'Vita Green ', 'VIPRE Security Group ', '3Cloud ', 'OffSec ', 'Asurion ', 'WebFX ', '3Cloud ', 'Security Bank Corporation ', '3Cloud ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Michael Page ', 'Vita Green ', 'Quantrics Enterprises Inc. ', 'Inspectorio ', 'Nextiva ', 'RealPage, Inc. ', 'Netrix Global ', 'OffSec ', 'GoTeam ', 'WebFX ', '3Cloud ', 'WebFX ', 'MultiplyMii ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Manila, National Capital Reg

Scraping the Job Description 37
['Infinera ', ' Philippines  2 weeks ago  ', ' 24 applicants']
Split details:  ['Philippines', '2', 'weeks ago']
Skills:  ['Skills: Quotations', ' Subsea Engineering', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', 'Vita Green ', 'VIPRE Security Group ', '3Cloud ', 'OffSec ', 'Asurion ', 'WebFX ', '3Cloud ', 'Security Bank Corporation ', '3Cloud ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Michael Page ', 'Vita Green ', 'Quantrics Enterprises Inc. ', 'Inspectorio ', 'Nextiva ', 'RealPage, Inc. ', 'Netrix Global ', 'OffSec ', 'GoTeam ', 'WebFX ', '3Cloud ', 'WebFX ', 'MultiplyMii ', '3Cloud ', 'Infinera ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philippines  8 minutes ago', 'Man

Scraping the Job Description 39
['Turing ', ' Philippines  5 days ago  ', ' 32 applicants']
Split details:  ['Philippines', '5', 'days ago']
Skills:  ['Skills: Front-End Development', ' TypeScript', ' +8 more']
['Sense Software Solutions ', 'Career Shepherd ', 'Quantrics Enterprises Inc. ', 'Eclaro ', 'Turing ', 'Career Shepherd ', 'Sourcepass ', 'Sourcepass ', 'Fingerprint for Success (F4S) ', 'Filta. ', 'Vita Green ', 'VIPRE Security Group ', '3Cloud ', 'OffSec ', 'Asurion ', 'WebFX ', '3Cloud ', 'Security Bank Corporation ', '3Cloud ', 'GoTeam ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Quantrics Enterprises Inc. ', 'Michael Page ', 'Vita Green ', 'Quantrics Enterprises Inc. ', 'Inspectorio ', 'Nextiva ', 'RealPage, Inc. ', 'Netrix Global ', 'OffSec ', 'GoTeam ', 'WebFX ', '3Cloud ', 'WebFX ', 'MultiplyMii ', '3Cloud ', 'Infinera ', 'Inspectorio ', 'Turing ']
['Philippines Reposted  2 weeks ago', 'Philippines Reposted', 'Manila, National Capital Region, Philipp

## Storing Collected Jobs Data in a CSV File

In [7]:
# Storing the data to a CSV file
df.to_csv('startup_jobs_entrylevel.csv', index=False)

# Output job descriptions to a TXT file
with open('startups_job_descriptions.txt', 'w', encoding="utf-8") as f:
    for line in job_desc:
        f.write(line)
        f.write('\n')

# Close the WebDriver
driver.quit()

In [9]:
df

,Job Title,Company Name,Company Location,Post Dates,Number of Applicants,Skills List,Job Description
0,PowerBI Developer,Sense Software Solutions,Philippines Reposted 2 weeks ago,2weeks ago,88 applicants,"[Qlik Sense, Dashboards]",About the job\nTechnical :\n4+ years in PowerB...
1,Front End Developer (For Active Pooling),Career Shepherd,Philippines Reposted,8minutes ago,1 applicant,"[Cascading Style Sheets (CSS), JavaScript]",About the job\nDISCLAIMER: This job posting is...
2,Golang Software Developer (Work from Home) + S...,Quantrics Enterprises Inc.,"Manila, National Capital Region, Philippines ...",23hours ago,16 applicants,"[Software Development, Creative Problem Solving]",About the job\nOverview\nJoin our award-winnin...
3,Applications DevOps Engineer (Zuora Consultant),Eclaro,"Manila, National Capital Region, Philippines",1week ago,5 applicants,"[Zuora, Validation Rules]",About the job\nApplications DevOps Engineer\n\...
4,Flutter Engineer,Turing,Philippines 23 hours ago,2months ago,27 applicants,[Computer Science],About the job\nA rapidly-growing company that ...
5,Flutter Developer (For Active Pooling),Career Shepherd,Philippines,1month ago,292 applicants,"[Mobile Application Development, REST APIs]",About the job\nDISCLAIMER: This job posting is...
6,iOS Mobile Application Developer,Sourcepass,"Quezon City, National Capital Region, Philippi...",1month ago,19 applicants,"[iOS Development, Swift (Programming Language)]",About the job\nAbout Sourcepass\n\nWe are Sour...
7,Android Mobile Application Developer,Sourcepass,"Quezon City, National Capital Region, Philippines",2months ago,125 applicants,"[Computer Science, Android Studio]",About the job\nAbout Sourcepass\n\nWe are Sour...
8,Expression of Interest: Mobile Application Dev...,Fingerprint for Success (F4S),Philippines 2 months ago,3weeks ago,42 applicants,"[Mobile Application Development, iOS Developm...",About the job\nWe are inviting professionals i...
9,Mid-level Frontend Developer – with Laptop & I...,Filta.,Philippines,4weeks ago,37 applicants,"[Front-End Development, Vanilla JavaScript]",About the job\n‘Invent better outcomes’ with t...
